- Họ tên: Võ Minh Tùng
- MSHV : 2370345

## Vision tranformer

install stanford card dataset keras

In [2]:
%pip install transformers==4.46.0



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install --upgrade tensorflow



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: "'transformers[flax]'": Expected package name at the start of dependency specifier
    'transformers[flax]'
    ^


In [5]:
%pip install tensorflow==2.11 keras==2.11

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.11 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for tensorflow==2.11


In [ ]:
import tensorflow as tf
import keras
import transformers
import sys

print("Python version:", sys.version)

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)
print("Transformers library version:", transformers.__version__)


Python version: 3.12.7 (tags/v3.12.7:0b05ead, Oct  1 2024, 03:06:41) [MSC v.1941 64 bit (AMD64)]
TensorFlow version: 2.18.0
Keras version: 3.5.0
Transformers library version: 4.46.0


Chuẩn bị Bộ Dữ Liệu Stanford Cars

In [5]:
import tensorflow as tf
from transformers import TFViTForImageClassification
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam

# Define dataset directory and parameters
data_dir = "./stanford_cars"
batch_size = 32
img_size = (224, 224)

# Load the training and validation datasets
train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
)

val_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
)

# Retrieve class names and set the number of classes
class_names = train_ds.class_names
num_classes = len(class_names)

# Define a preprocessing function
def preprocess(image, label):
    image = tf.image.resize(image, img_size)  # Resize to (224, 224)
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
    return image, label

# Apply preprocessing
train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)

# Debugging: Print data type and shape information
for images, labels in train_ds.take(1):
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)
    print("Image data type:", images.dtype)
    print("Label data type:", labels.dtype)






e:\software\python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Found 16185 files belonging to 2 classes.
Using 12948 files for training.
Found 16185 files belonging to 2 classes.
Using 3237 files for validation.
Image batch shape: (32, 224, 224, 3)
Label batch shape: (32,)
Image data type: <dtype: 'float32'>
Label data type: <dtype: 'int32'>


Create model

In [11]:

# Initialize a ViT model from the transformers library
model = TFViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224", attn_implementation="sdpa"
)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

All PyTorch model weights were used when initializing TFViTForImageClassification.

All the weights of TFViTForImageClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTForImageClassification for predictions without further training.


ValueError: Could not interpret optimizer identifier: <keras.src.optimizers.adam.Adam object at 0x000001C2F3BB3950>

In [7]:
model.summary()

Model: "tf_vi_t_for_image_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vit (TFViTMainLayer)        multiple                  85798656  
                                                                 
 classifier (Dense)          multiple                  769000    
                                                                 
Total params: 86567656 (330.23 MB)
Trainable params: 86567656 (330.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
for images, labels in train_ds.take(10):
    print(images.shape, labels.shape)


(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)


In [9]:
for images, labels in val_ds.take(10):
    print(images.shape, labels.shape)


(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)
(32, 224, 224, 3) (32,)


In [10]:

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

Epoch 1/5


ValueError: in user code:

    File "e:\software\python\Lib\site-packages\tf_keras\src\engine\training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "e:\software\python\Lib\site-packages\tf_keras\src\engine\training.py", line 1381, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "e:\software\python\Lib\site-packages\tf_keras\src\engine\training.py", line 1370, in run_step  **
        outputs = model.train_step(data)
    File "e:\software\python\Lib\site-packages\transformers\modeling_tf_utils.py", line 1672, in train_step
        y_pred = self(x, training=True)
    File "e:\software\python\Lib\site-packages\tf_keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\vomtu\AppData\Local\Temp\__autograph_generated_filex0zjeq7o.py", line 40, in tf__run_call_with_unpacked_inputs
        raise
    File "C:\Users\vom